In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import sys

- asos.com (user-agent,no igredients)
- zalando (user-agent)
- sephora 
- avon.com (kz- no ingredients, com has)
- monamie.kz (no igredients)
- feelunique.com [интересный сайт косметики](https://row.feelunique.com/p/Charlotte-Tilbury-Charlottes-Magic-Cream-50ml)
- паряться насчет состава [insta page](https://www.instagram.com/pure_beautystore/?igshid=ev5m6oyq06zy)

In [2]:
home_url = 'https://en.zalando.de'

In [3]:
category = 'beauty-skincare'

In [4]:
page_number = 1

https://en.zalando.de/beauty-skincare/?p=2

In [5]:
https_line = home_url +'/' + str(category) + '/?p=' + str(page_number)

In [6]:
https_line

'https://en.zalando.de/beauty-skincare/?p=1'

- надо браузерские headers дать, как-будто ты user-agent. 
[тут дают решение](https://stackoverflow.com/questions/47375153/requests-get-in-python-giving-connection-timeout-error)

In [7]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}

In [8]:
response = requests.get(https_line, headers=headers)
soup = BeautifulSoup(response.content,'lxml')

In [9]:
obj = soup.find('a', attrs = {'class':'cat_imageLink-OPGGa'})
obj.attrs['href']

'/maybelline-new-york-lash-sensational-serum-lash-serum-clear-mj331f01j-s11.html'

In [10]:
product_links = soup.findAll('a', attrs = {'class':'cat_imageLink-OPGGa'})
product_links = [link.attrs['href'] for link in product_links]

In [11]:
home_url

'https://en.zalando.de'

In [12]:
def get_page_links(category, page_number):
      
    beauty_link = home_url +'/'+ str(category) + '/?p=' + str(page_number)
  
    response = requests.get(beauty_link, headers=headers)
    
    if not response.ok:
        return response.status_code
    
    html = response.content
    soup = BeautifulSoup(html,'lxml')
    
    product_links = soup.findAll('a', attrs = {'class':'cat_imageLink-OPGGa'})
    product_links = [ home_url + link.attrs['href'] for link in product_links]
    
    return product_links

In [13]:
links = get_page_links(category,2)
links[:3]

['https://en.zalando.de/eco-by-sonya-extreme-exfoliate-glove-bath-and-body-tool-neutral-eca31j000-s11.html',
 'https://en.zalando.de/clinique-superdefense-cc-cream-spf30-40ml-cc-cream-light-2-cll31g01e-s11.html',
 'https://en.zalando.de/origins-drink-uptm-intensive-mask-75-ml-face-mask-orb31g037-s11.html']

In [14]:
links = get_page_links(category,1)
links[:3]

['https://en.zalando.de/maybelline-new-york-lash-sensational-serum-lash-serum-clear-mj331f01j-s11.html',
 'https://en.zalando.de/maybelline-bb-cream-bb-cream-mj331e000-s12.html',
 'https://en.zalando.de/rosental-organics-rose-queen-roller-skincare-tool-rosenquarz-rol31j001-j11.html']

In [15]:
df1 = pd.DataFrame(columns=['brand', 'name', 'price', 'rating', 'skin_type', 'ingredients'])

In [16]:
df1

,brand,name,price,rating,skin_type,ingredients


In [17]:
product_page = 'https://en.zalando.de/pixi-glow-tonic-250ml-toner-tonic-pik31g000-s11.html'

time.sleep(1)
try:
    response = requests.get(product_page, headers=headers)
    html = response.content
    soup = BeautifulSoup(html,'lxml')
except requests.exceptions.ConnectionError:
    status_code = "Connection refused. Too many requests from this ip"

id = z-vegas-pdp-props

#### Brand name

In [18]:
soup.find('span', attrs={'class':'A95iT1 pDVUjz nmA88J NNECXo AHAcbe x--xNS'}).text

'Pixi'

In [19]:
def get_brand(soup):
    brand = soup.find('span', attrs={'class':'A95iT1 pDVUjz nmA88J NNECXo AHAcbe x--xNS'})
    return "" if not brand else brand.text

In [20]:
new_product_page = links[3]
new_response = requests.get(new_product_page, headers=headers)
new_html = new_response.content
new_soup = BeautifulSoup(new_html,'lxml')

In [21]:
new_product_page

'https://en.zalando.de/pixi-glow-tonic-250ml-toner-tonic-pik31g000-s11.html'

In [22]:
get_brand(new_soup)

'Pixi'

#### Product name

In [23]:
soup.find('h1', attrs = {'class':'A95iT1 pDVUjz nmA88J FI5Jsp x--xNS Y6OuCD BicgmA'}).text

'GLOW TONIC 250ML - Toner'

In [24]:
def get_name(soup):
    name = soup.find('h1', attrs = {'class':'A95iT1 pDVUjz nmA88J FI5Jsp x--xNS Y6OuCD BicgmA'})
    return "" if not name else name.text

In [25]:
get_name(new_soup)

'GLOW TONIC 250ML - Toner'

#### Product price

In [26]:
text = soup.find('span', attrs = {'class':'A95iT1 pDVUjz nmA88J _0uQAcH AHAcbe A_8GRa Xb35xC'}).text
text

'23,95\xa0€'

In [27]:
pattern = re.compile(r'\d+\,?\d*')
result = re.match(pattern, text)

In [28]:
price = result.group(0)

In [29]:
float(price.replace(',','.'))

23.95

In [30]:
def get_price(soup):
    price_tag = soup.find('span', attrs = {'class':'A95iT1 pDVUjz nmA88J _0uQAcH AHAcbe A_8GRa Xb35xC'})
    if price_tag:
        price_phrase = price_tag.text
    else:
        discount = soup.find('span', attrs={'class':'A95iT1 pDVUjz nmA88J _0uQAcH AHAcbe gzB009'})
        price_phrase = discount.text
        
    pattern = re.compile(r'\d+\,?\d*')
    result = re.match(pattern, price_phrase)
    price = result.group(0)
    return float(price.replace(',','.'))

In [31]:
get_price(new_soup)

23.95

In [32]:
get_price(soup)

23.95

#### Skin type

In [33]:
product_details = soup.find('div', attrs = {'role':'tabpanel'})

In [34]:
tab_contents = product_details.findAll('div', attrs = {'class':'h-tabs__panel'})

In [35]:
highlights = tab_contents[0]

In [36]:
highlights_text = highlights.findAll('span', attrs = {'class':'body', 'as':'span'})

In [37]:
highlights_dict = {}

In [38]:
for i in range(0,len(highlights_text),2):
    clear_text = highlights_text[i].text.split(':')[0]
    highlights_dict[clear_text] = highlights_text[i+1].text

In [39]:
highlights_dict

{'Article number': 'PIK31G000-S11',
 'Concerns': 'Dryness, Dullness, Uneven Texture, Oiliness, Uneven skin tone, Acne',
 'Consistency': 'Liquid',
 'Skin type': 'All skin types',
 'Volume': '250 ml',
 'Warnings': 'We recommend using SPF in the daytime, especially when using a product with glycolic acid.'}

In [40]:
def get_skin_type(soup):
    product_details = soup.find('div', attrs = {'role':'tabpanel'})
    tab_contents = product_details.findAll('div', attrs = {'class':'h-tabs__panel'})
    highlights = tab_contents[0]
    highlights_text = highlights.findAll('span', attrs = {'class':'body', 'as':'span'})
    highlights_dict = {}
    for i in range(0,len(highlights_text),2):
        clear_text = highlights_text[i].text.split(':')[0]
        highlights_dict[clear_text] = highlights_text[i+1].text
        
    return "" if 'Skin type' not in highlights_dict else highlights_dict['Skin type']

In [41]:
get_skin_type(soup)

'All skin types'

In [42]:
page = 'https://en.zalando.de/pixi-glow-tonic-250ml-toner-tonic-pik31g000-s11.html'
response2 = requests.get(page, headers=headers)
html2= response2.content
soup2 = BeautifulSoup(html2,'lxml')

In [43]:
get_skin_type(new_soup)

'All skin types'

#### Ingredients

In [44]:
ingredients = tab_contents[1]

In [45]:
titles = ingredients.findAll('span', attrs = {'class':'h-text h-color-black body h-bold'})

In [46]:
titles[0].text

'Ingredients:\xa0'

In [47]:
ingredients_pos = -1 

In [48]:
for i in range(0, len(titles)):
    if 'Ingredients:' in titles[i].text:
        ingredients_pos = i 

In [49]:
ingredients_pos

0

In [50]:
contents = ingredients.find('p', attrs = {'class':'h-text h-color-black body h-m-bottom-xs'})

In [51]:
contents

<p as="p" class="h-text h-color-black body h-m-bottom-xs"><span as="span" class="h-text h-color-black body h-bold">Ingredients<!-- -->: </span><span as="span" class="h-text h-color-black body"><div class="h-container h-align-left"><p as="p" class="h-text h-color-black body">Aqua, Aloe Barbadensis Leaf Juice, Hamamelis Virginiana (Witch Hazel) Leaf Extract, Aesculus Hippocastanum (Horse Chestnut) Seed Extract, Glycolic Acid, Ammonium Glycolate, Glycerin, Butylene Glycol, Hexylene Glycol (and) Fructose (and) Glucose (a...</p><button as="button" class="h-action h-container--slide__link--black h-m-top-s -link underline"><span class="h-text h-color-black body -text-only"><h5 class="h-text h-color-black title-4">Discover more</h5></span></button></div></span></p>

In [52]:
contents.find('p', attrs = {'class':'h-text h-color-black body'}).text

'Aqua, Aloe Barbadensis Leaf Juice, Hamamelis Virginiana (Witch Hazel) Leaf Extract, Aesculus Hippocastanum (Horse Chestnut) Seed Extract, Glycolic Acid, Ammonium Glycolate, Glycerin, Butylene Glycol, Hexylene Glycol (and) Fructose (and) Glucose (a...'

In [54]:
def get_ingredients(soup):    
    product_details = soup.find('div', attrs = {'role':'tabpanel'})
    tab_contents = product_details.findAll('div', attrs = {'class':'h-tabs__panel'})
    ingredients = tab_contents[1]
    titles = ingredients.findAll('span', attrs = {'class':'h-text h-color-black body h-bold'})
    contents = ingredients.findAll('p', attrs = {'class':'h-text h-color-black body h-m-bottom-xs'})
    ingredients_pos = -1
    for i in range(0, len(titles)):
        if 'Ingredients:' in titles[i].text:
            ingredients_pos = i 
    return "" if ingredients_pos == -1 else contents[0].find('p', attrs = {'class':'h-text h-color-black body'}).text

In [55]:
get_ingredients(new_soup)

'Aqua, Aloe Barbadensis Leaf Juice, Hamamelis Virginiana (Witch Hazel) Leaf Extract, Aesculus Hippocastanum (Horse Chestnut) Seed Extract, Glycolic Acid, Ammonium Glycolate, Glycerin, Butylene Glycol, Hexylene Glycol (and) Fructose (and) Glucose (a...'

In [56]:
get_ingredients(soup2)

'Aqua, Aloe Barbadensis Leaf Juice, Hamamelis Virginiana (Witch Hazel) Leaf Extract, Aesculus Hippocastanum (Horse Chestnut) Seed Extract, Glycolic Acid, Ammonium Glycolate, Glycerin, Butylene Glycol, Hexylene Glycol (and) Fructose (and) Glucose (a...'

#### Rating

In [57]:
def get_average_rating(html_content):
    search_token = 'averageStarRating'
    offset = len(search_token) + 2
    start_pos = html_content.find(search_token)
    colon_pos = html_content.find(":", start_pos)
    comma_pos = html_content.find(",", start_pos)
    start_pos, colon_pos, comma_pos
    rating = html_content[colon_pos+1: comma_pos]
    return "" if rating=='null' else float(rating)

In [58]:
for l in links[:3]:
    html_content = str(requests.get(l, headers=headers).content)
    print(get_average_rating(html_content))
    time.sleep(1)

3.4285715
4.375
4.5789475


In [59]:
def get_product_data(product_page):
    """
        returns dictionary data_row
        
        product_page: string
            url for the product html page
    
    """    
    response = requests.get(product_page, headers=headers)
    
    if not response.ok:
        return response.status_code
    
    html = response.content
    soup = BeautifulSoup(html,'lxml')
    html_content = str(response.content)
    
    product_brand = get_brand(soup)
    product_name = get_name(soup)
    product_price = get_price(soup)
    
    product_rating = get_average_rating(html_content)
    ingredients = get_ingredients(soup)
    skin_type = get_skin_type(soup)


    data_row = {"brand":product_brand,
                "name":product_name, 
                "price":product_price, 
                "rating":product_rating,
                "skin_type":skin_type, 
                "ingredients":ingredients
               }

    return data_row

In [70]:
data_row = get_product_data(new_product_page)

In [71]:
df1 = df1.append(data_row, ignore_index=True)

In [72]:
df1

,brand,name,price,rating,skin_type,ingredients
0,Darphin,MELAPERFECT FOUNDATION NEUTRAL - Foundation,39.95,4,,"Water\aqua\eau, methyl, trimethicone, phenyl t..."
1,Nyx Professional Makeup,HD PHOTOGENIC CONCEALER WAND - Concealer,6.95,4.14365,,"AQUA / WATER, TRIMETHYLSILOXYPHENYL DIMETHICON..."
2,Nyx Professional Makeup,BARE WITH ME TINTED SKIN VEIL - Setting spray ...,9.95,4.56,,"Aqua/Water/Eau, Glycerin, Alcohol Denat., Aloe..."
3,MAC,PREP + PRIME FIX + MATTIFYING MIST 100ML - Primer,22.50,4.32759,,"Water\Aqua\Eau , Alcohol Denat. , Silica , Sod..."
4,MAC,STUDIO FIX FLUID SPF15 FOUNDATION - Foundation,33.95,4.48535,,"Octinoxate 2.50%, Titanium Dioxide 1.00%Water\..."
5,MAC,PREP + PRIME FIX +100ML - Primer,22.50,4.74413,,"Water\Aqua\Eau, Glycerin, Butylene Glycol, Cuc..."
6,MAC,PRO LONGWEAR CONCEALER - Concealer,23.50,4.47087,,"Cyclopentasiloxane, Water\Aqua\Eau, Trimethyls..."
7,MAC,MINERALIZE SKINFINISH - Highlighter,34.95,4.66187,,"Mica, Talc, Nylon-12, Dimethicone, Isopropyl P..."
8,MAC,MINI PREP + PRIME FIX +LITTLE M.A.C 30ML - Set...,10.95,4.52717,,"Water\Aqua\Eau, Glycerin, Butylene Glycol, Cuc..."
9,the Balm,LOU-MANIZER - Highlighter,22.95,4.66368,All skin types,"MICA, ISOEICOSANE, POLYETHYLENE, BORON NITRIDE..."


In [67]:
# %%time
# for product_link in links:
#     data_row = get_product_data(product_link)
#     df1 = df1.append(data_row, ignore_index=True)

In [68]:
#df1

In [69]:
df1 = pd.DataFrame(columns=['brand', 'name', 'price', 'rating','skin_type','ingredients'])
categories = ['beauty-complexion', 'beauty-skincare-face','beauty-sun-protection']
df2 = pd.DataFrame(columns=['category','url'])

for category in categories:
    for page_number in range(1, 60):
        product_links = get_page_links(category, page_number) 
        for link in product_links:
            data_row = {'category': category, 'url': link}
            df2 = df2.append(data_row, ignore_index=True)
            for i in range(5):
                try:
                    # пытаемся собрать немного даты
                    data_row = get_product_data(link)           
                    df1 = df1.append(data_row, ignore_index=True) 
                    time.sleep(0.5)
                    break
                except:
                    continue

        final_df = pd.concat([df2, df1], axis=1)
        if page_number//10:
            print(final_df.shape)
        
    final_df.to_csv('data/cosmetics_{}.csv'.format(category))
            
final_df.to_csv('data/cosmetics.csv')

(320, 8)
(352, 8)
(384, 8)
(416, 8)
(448, 8)
(480, 8)
(512, 8)
(544, 8)
(576, 8)
(608, 8)
(640, 8)
(672, 8)
(704, 8)
(736, 8)
(768, 8)
(800, 8)
(832, 8)
(864, 8)
(896, 8)
(928, 8)
(960, 8)
(992, 8)
(1024, 8)
(1056, 8)
(1088, 8)
(1120, 8)
(1152, 8)
(1184, 8)
(1216, 8)
(1248, 8)
(1280, 8)
(1312, 8)
(1344, 8)
(1376, 8)
(1408, 8)
(1440, 8)
(1472, 8)
(1504, 8)
(1536, 8)
(1568, 8)
(1600, 8)
(1632, 8)
(1664, 8)
(1696, 8)
(1728, 8)
(1760, 8)
(1792, 8)
(1824, 8)
(1856, 8)
(1888, 8)
(2208, 8)
(2240, 8)
(2272, 8)
(2304, 8)
(2336, 8)
(2368, 8)
(2400, 8)
(2432, 8)
(2464, 8)
(2496, 8)
(2528, 8)
(2560, 8)
(2592, 8)
(2595, 8)
(2627, 8)
(2635, 8)
(2654, 8)
(2670, 8)
(2689, 8)
(2708, 8)
(2726, 8)
(2744, 8)
(2761, 8)
(2780, 8)
(2799, 8)
(2818, 8)
(2839, 8)
(2859, 8)
(2880, 8)
(2906, 8)
(2931, 8)
(2957, 8)
(2981, 8)
(3008, 8)
(3036, 8)
(3065, 8)
(3094, 8)
(3123, 8)
(3152, 8)
(3184, 8)
(3216, 8)
(3248, 8)
(3280, 8)
(3312, 8)
(3344, 8)
(3376, 8)
(3408, 8)
(3440, 8)
(3472, 8)
(3504, 8)
(3824, 8)
(3856, 8)
(3